In this notebook, some augmentaion methos is introduced. And i will give some insight and product method in shopee compte. If you find usefule, please give upvote, thx.

- Image Augmentation
- Text Augmentaion
- Product Augmentaion

在本个notebook中，介绍了一些数据扩增方法。我也会在介绍使用的同时，结合shopee比赛给出一些建议。如果你感谢内容对你有帮助，请给我点赞，谢谢。


You can check my other notebooks:

- [Shopee Products Matching: Image Part [English+中文]](https://www.kaggle.com/finlay/shopee-products-matching-image-part-english)
- [Shopee Products Matching: Text Part [English+中文]](https://www.kaggle.com/finlay/shopee-products-matching-text-part-english)
- [Shopee Products Matching: BoF Part [English+中文]](https://www.kaggle.com/finlay/shopee-products-matching-bof-part-english)
- [Shopee Products Matching: Augment Part [English中文]](https://www.kaggle.com/finlay/shopee-products-matching-augment-part-english)
- [[Unsupervised] Image + Text Baseline in 20min](https://www.kaggle.com/finlay/unsupervised-image-text-baseline-in-20min)

In [ ]:
# DATA_PATH = '../input/'
DATA_PATH = '../input/shopee-product-matching/'

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2, matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import gc

In [ ]:
COMPUTE_CV = True

test = pd.read_csv(DATA_PATH + 'test.csv')
if len(test)>3: COMPUTE_CV = False
else: print('this submission notebook will compute CV score, but commit notebook will not')

# COMPUTE_CV = False

if COMPUTE_CV:
    train = pd.read_csv(DATA_PATH + 'train.csv')
    train['image'] = DATA_PATH + 'train_images/' + train['image']
    tmp = train.groupby('label_group').posting_id.agg('unique').to_dict()
    train['target'] = train.label_group.map(tmp)
#     train_gf = cudf.read_csv(DATA_PATH + 'train.csv')
else:
    train = pd.read_csv(DATA_PATH + 'test.csv')
    train['image'] = DATA_PATH + 'test_images/' + train['image']
#     train_gf = cudf.read_csv(DATA_PATH + 'test.csv')
    
print('train shape is', train.shape )
train.head()

# Image Augmentation

图像数据扩增

https://github.com/albumentations-team/albumentations

In [ ]:
import albumentations as A

def visualize(image):
    plt.figure(figsize=(6, 6))
    plt.axis('off')
    plt.imshow(image)

OpenCV reads an image in BGR format (so color channels of the image have the following order: Blue, Green, Red). Albumentations uses the most common and popular RGB image format. So when using OpenCV, we need to convert the image format to RGB explicitly.

In [ ]:
image = cv2.imread(train['image'].iloc[0])
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
visualize(image)

In [ ]:
transform = A.HorizontalFlip(p=1)
augmented_image = transform(image=image)['image']
visualize(augmented_image)

In [ ]:
transform = A.ShiftScaleRotate(p=1)
augmented_image = transform(image=image)['image']
visualize(augmented_image)

Define an augmentation pipeline using Compose, pass the image to it and receive the augmented image

In [ ]:
transform = A.Compose([
    A.RandomRotate90(),
    A.Transpose(),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.50, rotate_limit=45, p=.75),
    A.Blur(blur_limit=3),
    A.OpticalDistortion(),
    A.GridDistortion(),
    A.HueSaturationValue(),
])
augmented_image = transform(image=image)['image']
visualize(augmented_image)

In [ ]:
transform = A.Compose([
        A.RandomRotate90(),
        A.Flip(),
        A.Transpose(),
        A.OneOf([
            A.IAAAdditiveGaussianNoise(),
            A.GaussNoise(),
        ], p=0.2),
        A.OneOf([
            A.MotionBlur(p=.2),
            A.MedianBlur(blur_limit=3, p=0.1),
            A.Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
        A.OneOf([
            A.OpticalDistortion(p=0.3),
            A.GridDistortion(p=.1),
            A.IAAPiecewiseAffine(p=0.3),
        ], p=0.2),
        A.OneOf([
            A.CLAHE(clip_limit=2),
            A.IAASharpen(),
            A.IAAEmboss(),
            A.RandomBrightnessContrast(),            
        ], p=0.3),
        A.HueSaturationValue(p=0.3),
    ])
augmented_image = transform(image=image)['image']
visualize(augmented_image)

# Text Augmentation

* https://github.com/makcedward/nlpaug
* https://github.com/jasonwei20/eda_nlp/

In [ ]:
!pip install nlpaug

In [ ]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

## Swap character randomly¶


In [ ]:
aug = nac.RandomCharAug(action="swap")

In [ ]:
texts = train['title'].iloc[:10]

for text in texts:
    augmented_text = aug.augment(text)
    
    print('-'*20)
    print('Original Input:{}'.format(text))
    print('Agumented Output:{}'.format(augmented_text))

## Delete character randomly


In [ ]:
aug = nac.RandomCharAug(action="delete")

In [ ]:
texts = train['title'].iloc[:10]

for text in texts:
    augmented_text = aug.augment(text)
    
    print('-'*20)
    print('Original Input:{}'.format(text))
    print('Agumented Output:{}'.format(augmented_text))


## Word Augmenter

Use of word2vec (Mikolov et al., 2013), GloVe (Pennington et al., 2014), fasttext (Joulin et al., 2016), BERT(Devlin et al., 2018) and wordnet to insert and substitute similar word. Word2vecAug, GloVeAug and FasttextAug use word embeddings to find most similar group of words to replace original word. On the other hand, BertAug use language models to predict possible target word. WordNetAug use statistics way to find the similar group of words.

In [ ]:
text = train['title'].iloc[0]

# model_type: word2vec, glove or fasttext
aug = naw.WordEmbsAug(
    model_type='fasttext', model_path='../input/fasttext-wikinews/wiki-news-300d-1M.vec')
augmented_text = aug.augment(text)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

In [ ]:
for text in train['title'].iloc[:10]:
    augmented_text = aug.augment(text)
    print('-'*20)
    print('Original Input:{}'.format(text))
    print('Agumented Output:{}'.format(augmented_text))

## Contextual Word Embeddings Augmenter (BERT)

In [ ]:
# Augment French by BERT
aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', aug_p=0.1)
text = "Bonjour, J'aimerais une attestation de l'employeur certifiant que je suis en CDI."
augmented_text = aug.augment(text)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

In [ ]:
for text in train['title'].iloc[:10]:
    augmented_text = aug.augment(text)
    print('-'*20)
    print('Original Input:{}'.format(text))
    print('Agumented Output:{}'.format(augmented_text))

## TF-IDF Augmenter

In [ ]:
import re
import nlpaug.model.word_stats as nmw

def _tokenizer(text, token_pattern=r"(?u)\b\w\w+\b"):
    token_pattern = re.compile(token_pattern)
    return token_pattern.findall(text)

# Tokenize input
train_x_tokens = [_tokenizer(x) for x in train['title']]

# Train TF-IDF model
tfidf_model = nmw.TfIdf()
tfidf_model.train(train_x_tokens)
tfidf_model.save('.')

# Load TF-IDF augmenter
aug = naw.TfIdfAug(model_path='.', tokenizer=_tokenizer)

In [ ]:
texts = train['title'].iloc[:10]

for text in texts:
    augmented_text = aug.augment(text)
    
    print('-'*20)
    print('Original Input:{}'.format(text))
    print('Agumented Output:{}'.format(augmented_text))

## Contextual Word Embeddings for Sentence Augmenter

In [ ]:
# model_path: xlnet-base-cased or gpt2
aug = nas.ContextualWordEmbsForSentenceAug(model_path='xlnet-base-cased')
augmented_texts = aug.augment(text, n=3)

In [ ]:
texts = train['title'].iloc[:10]

for text in texts:
    augmented_text = aug.augment(text)
    
    print('-'*20)
    print('Original Input:{}'.format(text))
    print('Agumented Output:{}'.format(augmented_text))

# Product/Pair Augmentation

we random swap the image and title in same group to get new product.

将商品图像和title，重新组合得到新的商品。

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
aug_image_title_pairs = []
for df in train.groupby('label_group'):
    for idx1, img in enumerate(df[1]['image']):
        for idx2, title in enumerate(df[1]['title']):
            if idx1 == idx2:
                continue
            
            aug_image_title_pairs.append([img, title])
            
    break

In [ ]:
pd.DataFrame(aug_image_title_pairs, columns=['image', 'title'])

In [ ]:
df[1]

we see this group has 3 product, now we get 6 new product!